# DAIN Colab alpha

Credits to original Colab file: 
https://github.com/baowenbo/DAIN/issues/44

My fork:
https://github.com/styler00dollar/DAIN

Enhancement by Styler00Dollar aka "sudo rm -rf / --no-preserve-root#8353" on discord. Please do not run this command in your linux terminal. It's rather meant as a joke.

A simple guide:
- Copy the .ipynb-file to your drive.
- Create a folder inside of Google Drive named "DAIN"
- If you want to interpolate a png-sequence, put your files with xxxxx.png starting at 1 in content/DAIN/MiddleBurySet/other-data/Beanbags/. If the input is a video, then ffmpeg will automatically use that folder.
- Don't forget to configure the filename in Colab. You will notice that down below.
- The default configuration is 4x. Change the code yourself if you want something different. This alpha does not include an easy option.
- I DO NOT recommend to "run all". Run a cell and wait, then run the next cell.


Stuff that should be improved:
- Currently the output does have some random limitations due to my code. No more than 100 Frames will be processed. Output also does have a 4-digit limit. Don't try something too big. This is an alpha that should work on small stuff. I should fix that later.
- Gif (or RGBA24?) seems to be broken. A mkv-video file worked fine.
- A filename variable.
- People that look at my Code will notice that I use a Folder named "Beanbags". Why?? I don't know.
- Maybe temp-copy to .ipynb_checkpoints can be avoided.
- Changing ffmpeg ouptut commands, so new framerate will be inserted automatically.
- Handling png-sequence and video better. PNG should be able to be copied from drive directly into the correct folder.

In [1]:
# Connect Google Drive

from google.colab import drive
drive.mount('/content/gdrive')
model_dir = '/content/gdrive/My Drive/DAIN/'
#!rm -rf '{model_dir}'
#os.makedirs(model_dir, exist_ok=True)
!ls -ltra '{model_dir}'/..

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
total 3723083
-rw------- 1 root root    1560010 Sep 27 11:42 'Getting started.pdf'
drwx------ 2 root root       4096 Sep 27 11:43 'Colab Notebooks'
drwx------ 2 root root       4096 Oct  1 09:56  Vergleichsbilder
drwx------ 2 root root       4096 Oct  2 14:51  Temp
drwx------ 2 root root       4096 Nov 23 11:00  ESRGAN
-rw------- 1 root root        151 Nov 23 19:52  resize.gdoc
drwx------ 2 root root       4096 Nov 24 15:09  VapourSynthScripts
drwx------ 2 ro

In [1]:
# Standart ffmpeg
#!apt install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [66]:
# Check your current GPU
# If you are lucky, you get 16GB VRAM. If you are not lucky, you get less. VRAM is important. The more VRAM, the higher the maximum resolution will go.

# P100: 16GB (Can handle 720p. 1080p does not work and you will get an out-of-memory error.)
# P4: 8GB (Not tested.)

#%cd /tmp
#!youtube-dl -f 299 --postprocessor-args "-ss 0:0:1 -to 0:0:20" "https://www.youtube.com/watch?v=hU_NLZYUDyI"
!nvidia-smi --query-gpu=gpu_name --format=csv
#!wget -O "gdrive/My Drive/ESRGANModels/dejpeg1.pth" https://de-next.owncube.com/index.php/s/w82HLrLWmWi4SQ5/download
!nvidia-smi

name
Tesla P100-PCIE-16GB
Wed Mar  4 05:41:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                  

In [10]:
# Probably useless, but run it anyway. If it makes problems, then don't run it. Will check later.

#!git clone https://github.com/kkroening/ffmpeg-python/
#!cd ffmpeg-python
#!python setup.py
#!cd ..

#!pip install ffmpeg-python

!pip install opencv-python
!apt install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [0]:
 from IPython.display import clear_output

In [7]:
#!git clone https://github.com/baowenbo/DAIN.git

!git clone https://github.com/styler00dollar/DAIN.git

Cloning into 'DAIN'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 594 (delta 1), reused 0 (delta 0), pack-reused 587
Receiving objects: 100% (594/594), 186.71 KiB | 865.00 KiB/s, done.
Resolving deltas: 100% (329/329), done.


In [8]:
cd DAIN/my_package 

/content/DAIN/my_package


In [0]:
# This takes a while. Just wait. ~15 minutes.
!./build.sh

clear_output()

In [11]:
cd ../PWCNet/correlation_package_pytorch1_0

/content/DAIN/PWCNet/correlation_package_pytorch1_0


In [0]:
# Wait again. ~5 minutes.

!./build.sh

clear_output()

In [13]:
cd /content/DAIN

/content/DAIN


In [0]:
!mkdir model_weights MiddleBurySet

In [15]:
cd model_weights

/content/DAIN/model_weights


In [16]:
!wget http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth

--2020-03-04 04:57:43--  http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth
Resolving vllab1.ucmerced.edu (vllab1.ucmerced.edu)... 169.236.184.68
Connecting to vllab1.ucmerced.edu (vllab1.ucmerced.edu)|169.236.184.68|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96319643 (92M)
Saving to: ‘best.pth’

best.pth            100%[===================>]  91.86M  18.2MB/s    in 6.0s    

2020-03-04 04:57:50 (15.4 MB/s) - ‘best.pth’ saved [96319643/96319643]



In [17]:
cd ../MiddleBurySet

/content/DAIN/MiddleBurySet


In [18]:
!wget http://vision.middlebury.edu/flow/data/comp/zip/other-color-allframes.zip

--2020-03-04 04:57:58--  http://vision.middlebury.edu/flow/data/comp/zip/other-color-allframes.zip
Resolving vision.middlebury.edu (vision.middlebury.edu)... 140.233.20.14
Connecting to vision.middlebury.edu (vision.middlebury.edu)|140.233.20.14|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33671986 (32M) [application/zip]
Saving to: ‘other-color-allframes.zip’

other-color-allfram 100%[===================>]  32.11M  19.5MB/s    in 1.6s    

2020-03-04 04:58:00 (19.5 MB/s) - ‘other-color-allframes.zip’ saved [33671986/33671986]



In [0]:
!unzip other-color-allframes.zip

clear_output()

In [20]:
!wget http://vision.middlebury.edu/flow/data/comp/zip/other-gt-interp.zip

--2020-03-04 04:58:14--  http://vision.middlebury.edu/flow/data/comp/zip/other-gt-interp.zip
Resolving vision.middlebury.edu (vision.middlebury.edu)... 140.233.20.14
Connecting to vision.middlebury.edu (vision.middlebury.edu)|140.233.20.14|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4690196 (4.5M) [application/zip]
Saving to: ‘other-gt-interp.zip’

other-gt-interp.zip 100%[===================>]   4.47M  6.08MB/s    in 0.7s    

2020-03-04 04:58:15 (6.08 MB/s) - ‘other-gt-interp.zip’ saved [4690196/4690196]



In [0]:
!unzip other-gt-interp.zip

clear_output()

In [22]:
cd ..

/content/DAIN


In [0]:
!CUDA_VISIBLE_DEVICES=0

In [24]:
# Fix scipy
# DO NOT RESTART YOUR COLAB!! Yes, there is red text and a warning, but it works.

#!pip uninstall scipy
!pip install --force-reinstall scipy==1.0.0

     |████████████████████████████████| 50.0MB 62kB/s 
     |████████████████████████████████| 20.2MB 53.3MB/s 
ERROR: seaborn 0.10.0 has requirement scipy>=1.0.1, but you'll have scipy 1.0.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.0.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: cvxpy 1.0.25 has requirement scipy>=1.1.0, but you'll have scipy 1.0.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.17.5
    Uninstalling numpy-1.17.5:
      Successfully uninstalled numpy-1.17.5
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [67]:
# REMOVE OTHER-DATA & RESULT
!ls -a
!rm -rf ./MiddleBurySet/other-data/*
!rm -rf ./MiddleBurySet/other-result-author/*
!mkdir ./MiddleBurySet/other-data/Beanbags
!mkdir  '/content/DAIN/MiddleBurySet/other-result-author/TEST'
!mkdir  '/content/DAIN/MiddleBurySet/other-result-author/TEST/Beanbags'
#!unzip "./MiddleBurySet/other-gt-interp.zip" -d "./MiddleBurySet/"
#!unzip  "./MiddleBurySet/other-color-allframes.zip" -d "./MiddleBurySet/"
#!unzip "./MiddleBurySet/own_test.zip" -d "./MiddleBurySet/"

#!rm -rf ./MiddleBurySet/other-data/Beanbags/*


.				.git		    my_package
..				.gitignore	    networks
AverageMeter.py			.ipynb_checkpoints  PWCNet
balancedsampler.py		LICENSE		    __pycache__
checkpoints			loss_function.py    README.md
colab_MiddleBury_slowmotion.py	lr_scheduler.py     Resblock
datasets			MegaDepth	    S2D_models
demo_MiddleBury.py		MiddleBurySet	    Stack.py
demo_MiddleBury_slowmotion.py	model_weights	    train.py
environment.yaml		my_args.py
Archive:  ./MiddleBurySet/own_test.zip
  inflating: ./MiddleBurySet/other-data/Beanbags/00001.png  
  inflating: ./MiddleBurySet/other-data/Beanbags/00002.png  
  inflating: ./MiddleBurySet/other-data/Beanbags/00003.png  
  inflating: ./MiddleBurySet/other-data/Beanbags/00004.png  
  inflating: ./MiddleBurySet/other-data/Beanbags/00005.png  
  inflating: ./MiddleBurySet/other-data/Beanbags/00006.png  
  inflating: ./MiddleBurySet/other-data/Beanbags/00007.png  
  inflating: ./MiddleBurySet/other-data/Beanbags/00008.png  
  inflating: ./MiddleBurySet/other-data/Beanbags/00009

In [74]:
# IMPORTANT:
# If you want to input a png-sequence, just insert the png's with xxxx.png starting at 0001 in ./MiddleBurySet/other-data/Beanbags/ and skip this.
# If you want to input a video, then use this.

# ffmpeg extract
# DO NOT FORGET TO CONFIGURE THE FILENAME!!!!

# The first line in the output does tell you the FPS.


!cp "/content/gdrive/My Drive/DAIN/720p_test.mkv" "/content/DAIN/"

import cv2

cap = cv2.VideoCapture('/content/DAIN/720p_test.mkv')

fps = cap.get(cv2.CAP_PROP_FPS)
print("FPS:")
print(fps)

!ffmpeg -i '/content/DAIN/720p_test.mkv' './MiddleBurySet/other-data/Beanbags/%05d.png'
!ls -a "./MiddleBurySet/other-data/Beanbags/."

FPS:
30.0
ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-lib

In [75]:
# Copy Images to temp colab folder
!rm -rf ./MiddleBurySet/other-data/.ipynb_checkpoints/*
#!cp "./MiddleBurySet/other-data/Beanbags/." "./MiddleBurySet/other-data/.ipynb_checkpoints/"
#!cp "./MiddleBurySet/other-data/Beanbags/00001.png" "./MiddleBurySet/other-data/.ipynb_checkpoints/"
#!cp "./MiddleBurySet/other-data/Beanbags/00002.png" "./MiddleBurySet/other-data/.ipynb_checkpoints/"
#!cp "./MiddleBurySet/other-data/Beanbags/00003.png" "./MiddleBurySet/other-data/.ipynb_checkpoints/"
#!cp "./MiddleBurySet/other-data/Beanbags/00004.png" "./MiddleBurySet/other-data/.ipynb_checkpoints/"

!cp -a ./MiddleBurySet/other-data/Beanbags/. ./MiddleBurySet/other-data/.ipynb_checkpoints/

!ls -a "./MiddleBurySet/other-data/.ipynb_checkpoints/"

.	   00124.png  00249.png  00374.png  00499.png  00624.png  00749.png
..	   00125.png  00250.png  00375.png  00500.png  00625.png  00750.png
00001.png  00126.png  00251.png  00376.png  00501.png  00626.png  00751.png
00002.png  00127.png  00252.png  00377.png  00502.png  00627.png  00752.png
00003.png  00128.png  00253.png  00378.png  00503.png  00628.png  00753.png
00004.png  00129.png  00254.png  00379.png  00504.png  00629.png  00754.png
00005.png  00130.png  00255.png  00380.png  00505.png  00630.png  00755.png
00006.png  00131.png  00256.png  00381.png  00506.png  00631.png  00756.png
00007.png  00132.png  00257.png  00382.png  00507.png  00632.png  00757.png
00008.png  00133.png  00258.png  00383.png  00508.png  00633.png  00758.png
00009.png  00134.png  00259.png  00384.png  00509.png  00634.png  00759.png
00010.png  00135.png  00260.png  00385.png  00510.png  00635.png  00760.png
00011.png  00136.png  00261.png  00386.png  00511.png  00636.png  00761.png
00012.png  00137.png  0

In [76]:
# Interpolation
# Do not worry about "Interpolate 3 frames".
# 4x means 0.25. Number^-1.

!python colab_MiddleBury_slowmotion.py --netName DAIN_slowmotion --time_step 0.25

revise the unique id to a random numer 48686
Namespace(SAVED_MODEL=None, alpha=[0.0, 1.0], arg='./model_weights/48686-Wed-Mar-04-05:51/args.txt', batch_size=1, channels=3, ctx_lr_coe=1.0, datasetName='Vimeo_90K_interp', datasetPath='', dataset_split=97, debug=False, depth_lr_coe=0.001, dtype=<class 'torch.cuda.FloatTensor'>, epsilon=1e-06, factor=0.2, filter_lr_coe=1.0, filter_size=4, flow_lr_coe=0.01, force=False, log='./model_weights/48686-Wed-Mar-04-05:51/log.txt', lr=0.002, netName='DAIN_slowmotion', no_date=False, numEpoch=100, occ_lr_coe=1.0, patience=5, rectify_lr=0.001, save_path='./model_weights/48686-Wed-Mar-04-05:51', save_which=1, seed=1, time_step=0.25, uid=None, use_cuda=True, use_cudnn=1, weight_decay=0, workers=8)
cudnn is used
Interpolate 3 frames
The testing model weight is: ./model_weights/best.pth
The unique id for current testing is: TEST
.ipynb_checkpoints
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior 

In [89]:
# new_fps = fps * 4
# Automatic FPS calculation does not work. Just take your fps*4 and put it where "-r <number>" is.
!ffmpeg -r 120 -f image2 -i /content/DAIN/MiddleBurySet/other-result-author/TEST/%04d.png -vcodec libx264 -crf 25 -pix_fmt yuv420p test.mp4

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [0]:
# Copy result to drive
!cp /content/DAIN/test.mp4 '/content/gdrive/My Drive/DAIN/'

Code unter this text is "archived". Please, don't touch it.

In [54]:
# This is testcode. It does interpolate frame10.png and frame11.png.

#!cp "./MiddleBurySet/other-data/DogDance/frame10.png" "./MiddleBurySet/other-data/.ipynb_checkpoints/"
#!cp "./MiddleBurySet/other-data/DogDance/frame11.png" "./MiddleBurySet/other-data/.ipynb_checkpoints/"
#!ls -a "./MiddleBurySet/other-data/.ipynb_checkpoints/"

#!rm -rf ./MiddleBurySet/other-data/.ipynb_checkpoints/*
#!cp -a ./MiddleBurySet/other-data/Beanbags/. ./MiddleBurySet/other-data/.ipynb_checkpoints/
#!ls -a "./MiddleBurySet/other-data/.ipynb_checkpoints/"

#!python demo_MiddleBury_slowmotion.py --netName DAIN_slowmotion --time_step 0.25

 #clear_output()

.   00001.png  00003.png  00005.png  00007.png	  frame11.png
..  00002.png  00004.png  00006.png  frame10.png
revise the unique id to a random numer 63231
Namespace(SAVED_MODEL=None, alpha=[0.0, 1.0], arg='./model_weights/63231-Wed-Mar-04-05:32/args.txt', batch_size=1, channels=3, ctx_lr_coe=1.0, datasetName='Vimeo_90K_interp', datasetPath='', dataset_split=97, debug=False, depth_lr_coe=0.001, dtype=<class 'torch.cuda.FloatTensor'>, epsilon=1e-06, factor=0.2, filter_lr_coe=1.0, filter_size=4, flow_lr_coe=0.01, force=False, log='./model_weights/63231-Wed-Mar-04-05:32/log.txt', lr=0.002, netName='DAIN_slowmotion', no_date=False, numEpoch=100, occ_lr_coe=1.0, patience=5, rectify_lr=0.001, save_path='./model_weights/63231-Wed-Mar-04-05:32', save_which=1, seed=1, time_step=0.25, uid=None, use_cuda=True, use_cudnn=1, weight_decay=0, workers=8)
cudnn is used
Interpolate 3 frames
The testing model weight is: ./model_weights/best.pth
The unique id for current testing is: 82136
00003.png
Traceba

In [40]:
#  This notebok does not use it and you don't need to run it.

%%%writefile demo_MiddleBury.py
import time
import os
from torch.autograd import Variable
import math
import torch

import random
import numpy as np
import numpy
import networks
import cv2

from my_args import  args
from AverageMeter import  *

torch.backends.cudnn.benchmark = True # to speed up the


DO_MiddleBurryOther = True
MB_Other_DATA = "./MiddleBurySet/other-data/"
MB_Other_RESULT = "./MiddleBurySet/other-result-author/"
MB_Other_GT = "./MiddleBurySet/other-gt-interp/"
if not os.path.exists(MB_Other_RESULT):
    os.mkdir(MB_Other_RESULT)



model = networks.__dict__[args.netName](channel=args.channels,
                            filter_size = args.filter_size ,
                            timestep=args.time_step,
                            training=False)

if args.use_cuda:
    model = model.cuda()

args.SAVED_MODEL = './model_weights/best.pth'
if os.path.exists(args.SAVED_MODEL):
    print("The testing model weight is: " + args.SAVED_MODEL)
    if not args.use_cuda:
        pretrained_dict = torch.load(args.SAVED_MODEL, map_location=lambda storage, loc: storage)
        # model.load_state_dict(torch.load(args.SAVED_MODEL, map_location=lambda storage, loc: storage))
    else:
        pretrained_dict = torch.load(args.SAVED_MODEL)
        # model.load_state_dict(torch.load(args.SAVED_MODEL))

    model_dict = model.state_dict()
    # 1. filter out unnecessary keys
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    # 2. overwrite entries in the existing state dict
    model_dict.update(pretrained_dict)
    # 3. load the new state dict
    model.load_state_dict(model_dict)
    # 4. release the pretrained dict for saving memory
    pretrained_dict = []
else:
    print("*****************************************************************")
    print("**** We don't load any trained weights **************************")
    print("*****************************************************************")

model = model.eval() # deploy mode


use_cuda=args.use_cuda
save_which=args.save_which
dtype = args.dtype
unique_id =str(random.randint(0, 100000))
print("The unique id for current testing is: " + str(unique_id))

interp_error = AverageMeter()
if DO_MiddleBurryOther:
    subdir = os.listdir(MB_Other_DATA)
    gen_dir = os.path.join(MB_Other_RESULT, unique_id)
    os.mkdir(gen_dir)

    tot_timer = AverageMeter()
    proc_timer = AverageMeter()
    end = time.time()
    for dir in subdir:
        print(dir)
        os.mkdir(os.path.join(gen_dir, dir))
        arguments_strFirst = os.path.join(MB_Other_DATA, dir, "frame13.png")
        arguments_strSecond = os.path.join(MB_Other_DATA, dir, "frame14.png")
        arguments_strOut = os.path.join(gen_dir, dir, "frame10i11.png")
        gt_path = os.path.join(MB_Other_GT, dir, "frame10i11.png")

        X0 =  torch.from_numpy( np.transpose(cv2.imread(arguments_strFirst) , (2,0,1)).astype("float32")/ 255.0).type(dtype)
        X1 =  torch.from_numpy( np.transpose(cv2.imread(arguments_strSecond) , (2,0,1)).astype("float32")/ 255.0).type(dtype)


        y_ = torch.FloatTensor()

        assert (X0.size(1) == X1.size(1))
        assert (X0.size(2) == X1.size(2))

        intWidth = X0.size(2)
        intHeight = X0.size(1)
        channel = X0.size(0)
        if not channel == 3:
            continue

        if intWidth != ((intWidth >> 7) << 7):
            intWidth_pad = (((intWidth >> 7) + 1) << 7)  # more than necessary
            intPaddingLeft =int(( intWidth_pad - intWidth)/2)
            intPaddingRight = intWidth_pad - intWidth - intPaddingLeft
        else:
            intWidth_pad = intWidth
            intPaddingLeft = 32
            intPaddingRight= 32

        if intHeight != ((intHeight >> 7) << 7):
            intHeight_pad = (((intHeight >> 7) + 1) << 7)  # more than necessary
            intPaddingTop = int((intHeight_pad - intHeight) / 2)
            intPaddingBottom = intHeight_pad - intHeight - intPaddingTop
        else:
            intHeight_pad = intHeight
            intPaddingTop = 32
            intPaddingBottom = 32

        pader = torch.nn.ReplicationPad2d([intPaddingLeft, intPaddingRight , intPaddingTop, intPaddingBottom])

        torch.set_grad_enabled(False)
        X0 = Variable(torch.unsqueeze(X0,0))
        X1 = Variable(torch.unsqueeze(X1,0))
        X0 = pader(X0)
        X1 = pader(X1)

        if use_cuda:
            X0 = X0.cuda()
            X1 = X1.cuda()
        proc_end = time.time()
        y_s,offset,filter = model(torch.stack((X0, X1),dim = 0))
        y_ = y_s[save_which]

        proc_timer.update(time.time() -proc_end)
        tot_timer.update(time.time() - end)
        end  = time.time()
        print("*****************current image process time \t " + str(time.time()-proc_end )+"s ******************" )
        if use_cuda:
            X0 = X0.data.cpu().numpy()
            y_ = y_.data.cpu().numpy()
            offset = [offset_i.data.cpu().numpy() for offset_i in offset]
            filter = [filter_i.data.cpu().numpy() for filter_i in filter]  if filter[0] is not None else None
            X1 = X1.data.cpu().numpy()
        else:
            X0 = X0.data.numpy()
            y_ = y_.data.numpy()
            offset = [offset_i.data.numpy() for offset_i in offset]
            filter = [filter_i.data.numpy() for filter_i in filter]
            X1 = X1.data.numpy()



        X0 = np.transpose(255.0 * X0.clip(0,1.0)[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0))
        y_ = np.transpose(255.0 * y_.clip(0,1.0)[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0))
        offset = [np.transpose(offset_i[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0)) for offset_i in offset]
        filter = [np.transpose(
            filter_i[0, :, intPaddingTop:intPaddingTop + intHeight, intPaddingLeft: intPaddingLeft + intWidth],
            (1, 2, 0)) for filter_i in filter]  if filter is not None else None
        X1 = np.transpose(255.0 * X1.clip(0,1.0)[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0))


        cv2.imwrite(arguments_strOut, np.round(y_).astype(numpy.uint8))


        rec_rgb =  cv2.imread(arguments_strOut)
        gt_rgb = cv2.imread(gt_path)

        diff_rgb = 128.0 + rec_rgb - gt_rgb
        avg_interp_error_abs = np.mean(np.abs(diff_rgb - 128.0))

        interp_error.update(avg_interp_error_abs, 1)

        mse = numpy.mean((diff_rgb - 128.0) ** 2)

        PIXEL_MAX = 255.0
        psnr = 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

        print("interpolation error / PSNR : " + str(round(avg_interp_error_abs,4)) + " / " + str(round(psnr,4)))
        metrics = "The average interpolation error / PSNR for all images are : " + str(round(interp_error.avg, 4))
        print(metrics)



Overwriting demo_MiddleBury.py


In [0]:
#current path
#!pwd

/content/DAIN


In [0]:
#create result zip
#!zip -r myzip "./MiddleBurySet/other-result-author/"

  adding: MiddleBurySet/other-result-author/ (stored 0%)
  adding: MiddleBurySet/other-result-author/.ipynb_checkpoints/ (stored 0%)
  adding: MiddleBurySet/other-result-author/TEST/ (stored 0%)
  adding: MiddleBurySet/other-result-author/TEST/0014.png (deflated 0%)
  adding: MiddleBurySet/other-result-author/TEST/0002.png (deflated 0%)
  adding: MiddleBurySet/other-result-author/TEST/0005.png (deflated 0%)
  adding: MiddleBurySet/other-result-author/TEST/0012.png (deflated 0%)
  adding: MiddleBurySet/other-result-author/TEST/0004.png (deflated 0%)
  adding: MiddleBurySet/other-result-author/TEST/0001.png (deflated 0%)
  adding: MiddleBurySet/other-result-author/TEST/0003.png (deflated 0%)
  adding: MiddleBurySet/other-result-author/TEST/0006.png (deflated 0%)
  adding: MiddleBurySet/other-result-author/TEST/0011.png (deflated 0%)
  adding: MiddleBurySet/other-result-author/TEST/Beanbags/ (stored 0%)
  adding: MiddleBurySet/other-result-author/TEST/0010.png (deflated 0%)
  adding: Midd